<a href="https://colab.research.google.com/github/rcarcelle/qualentum/blob/main/Desarrollo%20de%20Soluciones%20en%20IA%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import os
from datetime import datetime

# Configurar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# Crear carpeta para guardar imágenes
output_dir = "imagenes_generadas"
os.makedirs(output_dir, exist_ok=True)


In [3]:
# Scheduler
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base",
    subfolder="scheduler"
)

# Pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base",
    scheduler=scheduler,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
pipe = pipe.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


scheduler_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
prompt = "A high-resolution product photo of modern running sneakers on a white background"
negative_prompt = "low quality, blurry, distorted, watermark"

image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=8.5,
    num_inference_steps=50,
).images[0]

# Guardar imagen
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
image_path = os.path.join(output_dir, f"sneakers_{timestamp}.png")
image.save(image_path)
image.show()


  0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
prompts = [
    "A luxury leather handbag with gold accents, studio lighting, white background",
    "A minimalist black smartwatch with metallic strap, top view",
    "Elegant sunglasses with reflection, isolated on white background"
]

for p in prompts:
    img = pipe(
        prompt=p,
        negative_prompt=negative_prompt,
        guidance_scale=7.5,
        num_inference_steps=40,
    ).images[0]

    filename = p.split()[2] + "_" + datetime.now().strftime("%H%M%S") + ".png"
    img.save(os.path.join(output_dir, filename))


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
# Cargar imagen base
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/cat.png"
init_image = load_image(url).resize((512, 512))

# Cargar pipeline de image-to-image
pipe_img2img = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

# Prompt para variar la imagen
prompt = "A futuristic sci-fi version of the same cat, glowing eyes, cyberpunk style"

# Generar variación
edited = pipe_img2img(
    prompt=prompt,
    image=init_image,
    strength=0.7,
    guidance_scale=8.0,
    num_inference_steps=40
).images[0]

# Guardar y mostrar
init_image.save(os.path.join(output_dir, "cat_original.png"))
edited.save(os.path.join(output_dir, "cat_variation.png"))

grid = make_image_grid([init_image, edited], rows=1, cols=2)
grid.show()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]